### Sentiment Analysis PREDICTION - Version 02

+ TFIDF + VADER + NEGATIVEWORD

+ Multidata review source, including VIB, TCB, CTG, etc.

In [1]:
import json
import pandas as pd
import numpy as np

from tqdm import tqdm
from string import punctuation
import seaborn as sns
import matplotlib.pyplot as plt
from itertools import groupby
import sys

import regex as re
from deep_translator import GoogleTranslator
from collections import Counter
import textacy.preprocessing.normalize as tprep
from textacy.preprocessing.remove import accents
import string

import nltk
from nltk.corpus import opinion_lexicon,wordnet,stopwords
from nltk.tokenize import word_tokenize, WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer

import os
from datetime import date
import datetime
import time
import shutil
import os

import enchant
dict_english = enchant.Dict("en_US")

#from TopicModeling import *
import pickle

# Android
from google_play_scraper import Sort, reviews, app, reviews_all

# iOS
from app_store_scraper import AppStore

In [2]:
# included code
import sys
sys.path.append('./mylib/')
import global_materials
from load import *
from sentiment_process import *

stopwords = global_materials.stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nhan.ngothanh1\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     C:\Users\nhan.ngothanh1\AppData\Roaming\nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\nhan.ngothanh1\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\nhan.ngothanh1\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nhan.ngothanh1\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\nhan.ngothanh1\AppData\Roaming\nlt

In [3]:
# PARAMETER
GET_SENTIMENT_OF_REVIEW_FROM_DATE = datetime.datetime(2022,9,1)

In [4]:
today = date.today()
PRED_ROOT = './data/predict'

if not os.path.exists("model"):
    os.mkdir("model")
if not os.path.exists("data"):
    os.mkdir("data")
if not os.path.exists(PRED_ROOT):
    os.mkdir(PRED_ROOT)  
if not os.path.exists(PRED_ROOT+"/translate_"+str(today)):
    os.mkdir(PRED_ROOT+"/translate_"+str(today))

________________________________________________________________________________________________________________________
### Data Input

You can get data input from any sources

This example you get data input from App [Android (GooglePlay) and iOS (Appstore)] and check sentiment of new reviews.

In [27]:

rw_android_src_dict = {'vib1':['com.vn.vib.mobileapp','vi','vn'],
                       'vib2':['com.vib.myvib2','vi','vn'],
                       'tcb':['vn.com.techcombank.bb.app','vi','vn'],
                       'tpb':['com.tpb.mb.gprsandroid','vi','vn'],
                       'bid':['vn.com.bidv.ibank','vi','vn'],
                       'vcb':['com.VCB','vi','vn'],
                       'mb':['com.mbcorp','vi','vn']
                       }
#----------------------------------------------------------------------------------------------------------
# get Android review
column = ['reviewId','userName','userImage','content','score','thumbsUpCount','reviewCreatedVersion','at','replyContent','repliedAt','bankapp']
df_android_rw = pd.DataFrame(columns = column)

get_review_in = ['vib1','vib2'] #rw_android_src_dict.keys()

for bank_name in get_review_in:
    Andrw_df = get_rwdata_android(rw_android_src_dict, bank_name, save_path=PRED_ROOT)   
    Andrw_df['bankapp'] = bank_name
    df_android_rw = pd.concat([df_android_rw, Andrw_df])
    time.sleep(30)
    
df_android_rw.reset_index(drop=True, inplace=True)
df_android_rw.to_csv(PRED_ROOT+'/1_AndRaw_'+str(today)+'vibx.csv', index=False)

print(df_android_rw.shape)
df_android_rw.head(1)

#----------------------------------------------------------------------------------------------------------
df_android_rw['month'] = pd.DatetimeIndex(df_android_rw['at']).month
df_android_rw['year'] = pd.DatetimeIndex(df_android_rw['at']).year
df_android_rw['m_y'] = df_android_rw['month'].astype(str)+df_android_rw['year'].astype(str)

#JUST RAW
df_android_used = df_android_rw[['userName','content','at','m_y','score']]
df_android_used.rename(columns = {'userName':'username','content':'review','at':'date','score':'rating'}, inplace=True)

# Convert timetamps to string
df_android_used['date'] = df_android_used['date'].apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))
print(df_android_used.shape)
df_android_used.head(1)

vib1: (4683, 10)
vib2: (1681, 10)
(6364, 11)
(6364, 5)


C:\Users\nhan.ngothanh1\AppData\Local\Temp\ipykernel_15972\206084083.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_android_used.rename(columns = {'userName':'username','content':'review','at':'date','score':'rating'}, inplace=True)
C:\Users\nhan.ngothanh1\AppData\Local\Temp\ipykernel_15972\206084083.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_android_used['date'] = df_android_used['date'].apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))


,username,review,date,m_y,rating
0,Trang web: Vay9.C0M,Vô trang 𝐕𝐚𝐲𝟗.𝐂𝟎𝐌 vay 0% trong 9tháng,2022-09-27 14:29:00,92022,1


In [28]:

rw_ios_src_dict = {'vib1':['MyVIB','vn','949371011'],
                   'vib2':['MyVIB 2.0','vn','1626624790'],
                   'tcb':['Techcombank Mobile','vn','1548623362'],
                   'tpb':['TPBank Mobile','vn','450464147'],
                   'bid':['SmartBanking','vn','1061867449'],
                   'vcb':['Vietcombank','vn','561433133'],
                   'mb':['BIZ MBBANK','vn','1424005630']
                  }

#----------------------------------------------------------------------------------------------------------
column = ['title','userName','isEdited','review','date','rating','developerResponse','bankapp']
df_ios_rw = pd.DataFrame(columns = column)

get_review_in = ['vib1','vib2'] #rw_ios_src_dict.keys()

for bankapp in get_review_in:
    iosrw_df = get_rwdata_ios(rw_ios_src_dict, bankapp, save_path=PRED_ROOT)   
    iosrw_df['bankapp'] = bankapp
    df_ios_rw = pd.concat([df_ios_rw, iosrw_df])  
    time.sleep(30)
    
today = date.today()
df_ios_rw.reset_index(drop=True, inplace=True)
df_ios_rw.to_csv(PRED_ROOT+'/1_iOSRaw_'+str(today)+'vibx.csv', index=False)

print(df_ios_rw.shape)

#----------------------------------------------------------------------------------------------------------
df_ios_rw['month'] = pd.DatetimeIndex(df_ios_rw['date']).month
df_ios_rw['year'] = pd.DatetimeIndex(df_ios_rw['date']).year
df_ios_rw['m_y'] = df_ios_rw['month'].astype(str)+df_ios_rw['year'].astype(str)

#JUST RAW
df_ios_used = df_ios_rw[['userName','review','date','m_y','rating']]
df_ios_used.rename(columns = {'userName':'username'}, inplace=True)

# Convert timetamps to string
df_ios_used['date'] = df_ios_used['date'].apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))



2022-09-28 17:52:54,429 [INFO] Base - Initialised: AppStore('vn', 'myvib', 949371011)
2022-09-28 17:52:54,429 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/vn/app/myvib/id949371011
2022-09-28 17:52:59,706 [INFO] Base - [id:949371011] Fetched 240 reviews (240 fetched in total)
2022-09-28 17:53:05,436 [INFO] Base - [id:949371011] Fetched 440 reviews (440 fetched in total)
2022-09-28 17:53:10,939 [INFO] Base - [id:949371011] Fetched 720 reviews (720 fetched in total)
2022-09-28 17:53:16,400 [INFO] Base - [id:949371011] Fetched 980 reviews (980 fetched in total)
2022-09-28 17:53:21,584 [INFO] Base - [id:949371011] Fetched 1248 reviews (1248 fetched in total)


vib1: (1248, 7)


2022-09-28 17:53:52,319 [INFO] Base - Initialised: AppStore('vn', 'myvib-2-0', 1626624790)
2022-09-28 17:53:52,319 [INFO] Base - Ready to fetch reviews from: https://apps.apple.com/vn/app/myvib-2-0/id1626624790
2022-09-28 17:53:57,512 [INFO] Base - [id:1626624790] Fetched 240 reviews (240 fetched in total)
2022-09-28 17:54:00,631 [INFO] Base - [id:1626624790] Fetched 394 reviews (394 fetched in total)


vib2: (394, 7)
(1642, 8)


C:\Users\nhan.ngothanh1\AppData\Local\Temp\ipykernel_15972\2850502776.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ios_used.rename(columns = {'userName':'username'}, inplace=True)
C:\Users\nhan.ngothanh1\AppData\Local\Temp\ipykernel_15972\2850502776.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ios_used['date'] = df_ios_used['date'].apply(lambda x: x.strftime("%Y-%m-%d %H:%M:%S"))


In [29]:
df_used = pd.concat([df_android_used, df_ios_used])
df_used['duplicate'] = df_used.duplicated(subset=['username', 'm_y', 'review'], keep='first')
print('BEFORE: df_used.shape = ',df_used.shape[0])
df_used = df_used[df_used['duplicate']==False] #remove duplicate
df_used.reset_index(inplace=True, drop=True)
print('AFTER DUPLICATE PROCESSING: df_used.shape = ',df_used.shape[0])

# process for date column
df_used['date'] = df_used['date'].apply(lambda x: ('20'+x) if (len(x)==17) else x ) # add '20' before datetime string for right format yyyy-mm-dd 
df_used['date'] = df_used['date'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d %H:%M:%S'))

df_used.to_csv(PRED_ROOT+'/2_dataApp_combine.csv',index=False,sep='\t')

BEFORE: df_used.shape =  8006
AFTER DUPLICATE PROCESSING: df_used.shape =  7754


In [5]:
df_used = pd.read_csv(PRED_ROOT+'/2_dataApp_combine.csv',index_col=False,sep='\t')
df_used['date'] = pd.to_datetime(df_used['date'])

__________________________________________________________________________________________________________________________
### Data Processing and Prediction

Vietnamese process --> English translation --> Clean text --> extract TFIDF + VADER + NEGWORD feature --> PREDICTION

In [6]:
df_used['ymdhms'] = pd.to_datetime(df_used['date']).dt.strftime("%Y_%m_%d_%H_%M_%S")

# Get the piece of data the you would like to predict.
df_new = df_used[df_used['date']>=GET_SENTIMENT_OF_REVIEW_FROM_DATE].reset_index(drop=True)

# RUN SENTIMENT PREDITION
df_pred_result = sentiment_prediction(df_new, tfidf_convertor='./model/TFIDF_VECTORIZER_app.pk', sentiment_model='./model/SENTIMENT_MODEL_svc_app.sav', pred_out_file=PRED_ROOT+'/OUT_sentiment_prediction.csv')
df_pred_result[['review_adj','review_en','review_clean','vader_compound','sentiment']]

There are 136 new reviews


,review_adj,review_en,review_clean,vader_compound,sentiment
0,vô trang vay trong,no loan page in,loan page,0,0
1,lừa đảo khách hàng chiếm thông tin cá nhân ký ...,scam customers to take personal information di...,scam customer take personal information digita...,1,1
2,vào trang 𝐕𝐚𝐲𝐓𝐨𝐭𝐍𝐡𝐚𝐭 vay và free,go to the page loan and free,go page loan free,0,0
3,vô trang 𝐕𝐚𝐲𝐓𝐨𝐭𝐍𝐡𝐚𝐭 để vay được nhận,Go to the page to get a loan,go page get loan,0,0
4,sao không cho đăng ký vậy,why not register?,not register,0,1
...,...,...,...,...,...
131,đưa ra nhiều chương trình ưu đãi miễn phí thườ...,offer many annual free incentive programs with...,offer many annual free incentive program uncle...,0,1
132,lỗi quá nhiều,too many errors,many error,1,1
133,một cái app ngân hàng ngu chưa từng thấy,a stupid banking app ever,stupid banking ever,1,1
134,mình có đổi otp rồi khi chuyển tiền có nhập đú...,I changed my otp and when I transferred the mo...,change transfer money enter correctly always r...,1,1


In [7]:
avg_sentiment_score = df_pred_result['sentiment'].sum()/df_pred_result.shape[0]
sentiment_score = -(avg_sentiment_score-0.5)*2
print(f"Neg/Pos ratio: {df_pred_result['sentiment'].sum()}/{df_pred_result.shape[0]-df_pred_result['sentiment'].sum()}")
print(f"Sentiment Score: {sentiment_score:.5f}")


Neg/Pos ratio: 89/44
Sentiment Score: -0.33835


In [11]:
pd.options.display.max_colwidth = 220
df_pred_result[df_pred_result['sentiment']==1][['review_adj','vader_compound','label_negword']][75:100]

,review_adj,vader_compound,label_negword
120,ông nào chưa test kĩ đã deploy prod ngày rồi không vào được,1,1
122,lỗi liên tọi đổi mật khẩu cả tiếng đồng hồ không xong,1,1
123,app này giao diện quá xấu thua app cũ nói chung là nhìn chẳng muốn xài nên học hỏi app của vcb,1,1
125,quá chậm,0,1
126,ứng dụng quá tệ thường xuyên lag giao dịch chuyển tiền lỗi mình bị trừ tiền bên kia không nhận được chờ đến ngày thứ rồi tiền không được hoàn về và người nhận vẫn không nhận được tiền,1,1
127,chả hiểu kiểu gì chuyển tiền tiền trừ mà thông báo hay lịch sử đều không có,0,1
128,đổi pin otp xong không xài được luôn báo sai pin mặc dù nhập đúng,1,1
129,nâng cấp lên làm gì không biết vô web không được sao kê giao dịch cũng không được,0,1
130,app mới như không được có mỗi cái tính năng chuyển tiền mà thao tác quá nhiều bước đã vậy còn không lưu được danh bạ để chuyển tiền đã xài vib năm chắc phải đổi sang app khác thôi,1,1
131,đưa ra nhiều chương trình ưu đãi miễn phí thường niên với các điều khoản không rõ ràng đánh đố trình độ hiểu biết của khách hàng,0,1


### END
______________________________________________________________________________________________________________________________________________________________________________________

In [32]:
sys.exit()

SystemExit: 

c:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


### [PHUONG] SENTIMENT PREDICTION for "Interal Rating" data 


In [ ]:
import cx_Oracle as ora

# Query data from DEC
dsnStr = ora.makedsn(host = '10.60.54.12', port = '1521', service_name = 'DEC_DATA')
con = ora.connect(user = 'ADM42244NHANNT', password = 'Vibhn321#!', dsn = dsnStr)
c = con.cursor()
result = c.execute('select * from dec_bu.u_raw_internal_rating_info').fetchall()
df_ir = pd.DataFrame(result)
df_ir.columns = [x[0] for x in c.description]

# fill Nan Value
df_ir['RATING_COMMENT'] = df_ir['RATING_COMMENT'].apply(lambda x: 0 if (x=='nan') else x)

# Get comment having content
df_w = df_ir[df_ir['RATING_COMMENT']!=0]
#df_w = df_w[['USER_NAME','RATING_COMMENT','RATING_POINT']]
df_w = df_w.rename(columns={'USER_NAME':'username','RATING_COMMENT':'review','RATING_POINT':'rating'})
print(df_w.shape[0])

# Run Sentiment Prediction
df_pred_result = sentiment_prediction(df_w, tfidf_convertor='./model/TFIDF_VECTORIZER_app.pk', sentiment_model='./model/SENTIMENT_MODEL_mnb_app.sav', pred_out_file='./internal_rating_sentiment.csv')
df_pred_result[['review','review_en','review_clean','vader_compound','sentiment']]

4233
There are 4233 new reviews
